In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed, BitsAndBytesConfig
import torch, json, gc, time
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
import time
import re
from diffusers import BitsAndBytesConfig, SD3Transformer2DModel
from diffusers import StableDiffusion3Pipeline
import os
os.environ['HF_TOKEN']="***"

# API KEY 정보로드
load_dotenv()

True

In [2]:
torch.cuda.empty_cache()
gc.collect()

20

In [3]:
# from datasets import load_dataset

# ds = load_dataset("abisee/cnn_dailymail", "3.0.0")
# gold_data = ds['train'].select(range(50))
 
# df = pd.DataFrame({
#     "article": gold_data["article"]
# })
# df.to_csv("sample_cnn_dailymail.csv", index=False, encoding="utf-8")

In [3]:
df = pd.read_csv('/home/user/Desktop/NewsToCartoon/sample_cnn_dailymail.csv')

In [4]:
# 출력 설정 조정
pd.set_option('display.max_colwidth', None)  # 열 너비를 자동으로 조정
pd.set_option('display.max_rows', None)      # 최대 행 수 설정 (기본값으로 모든 행 표시)
pd.set_option('display.expand_frame_repr', False)  # 한 화면에 다 표시

print(df)

In [5]:
set_seed(1234)

# models = {"Llama-3.1-8B":"meta-llama/Llama-3.1-8B-Instruct", "Ministral-8B-Instruct": "mistralai/Ministral-8B-Instruct-2410"}

# Define the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

In [6]:
# def generate_response(system_message, user_message, tokenizer, model):
#     messages = [
#         {"role": "system", "content": system_message},
#         {"role": "user", "content": user_message},
#     ]

#     input_ids = tokenizer.apply_chat_template(
#         messages,
#         add_generation_prompt=True,
#         return_tensors="pt",
#     ).to(model.device)  # 양자화된 모델 사용

#     # attention_mask 생성
#     pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
    
#     attention_mask = (input_ids != pad_token_id).long()

#     terminators = [
#         tokenizer.eos_token_id,
#         tokenizer.convert_tokens_to_ids("<|eot_id|>")
#     ]

#     outputs = model.generate(
#         input_ids,
#         attention_mask=attention_mask,
#         max_new_tokens=512,
#         eos_token_id=terminators,
#         do_sample=True,
#         temperature=0,
#         top_p=0.3
#     )

#     response = outputs[0][input_ids.shape[-1]:]
#     return tokenizer.decode(response, skip_special_tokens=True)


In [7]:
# loaded_models = {}
# for model_name, model_id in models.items():
#     print(f"Loading {model_name}...")
#     tokenizer = AutoTokenizer.from_pretrained(model_id)
#     model = AutoModelForCausalLM.from_pretrained(
#         model_id,
#         quantization_config=bnb_config,
#         device_map="cuda"
#     )
#     loaded_models[model_name] = (tokenizer, model)

In [59]:
storyboard_prompt = """
Analyze the input(article) to extract the 5W1H (Who, What, When, Where, Why, How) information, and use it to create a storyboard for a comic. 
Each sentence in the storyboard will be used as input for Stable Diffusion. Follow the conditions below:

### Instructions:
1. Clearly and concisely identify each element of the 5W1H based on the article content. If any information is missing, write "Unknown."
2. Use only the extracted 5W1H information to create the storyboard. Do not infer or add details beyond the provided information.
3. The storyboard must consist of exactly 6 scenes.  
4. Each scene description must be focus on and describe only action for drawing. 
5. Do not use pronouns in the sentences. Always use full names for people or objects.  
6. Do not use commas, single quotes, or double quotes in the sentences.  
7. Each sentence should be lower than 77 tokens and detailed enough for visual representation.  
8. The output only include the storyboard.  

---

### Output Format:  
"Scene 1": "Scene description sentence",
"Scene 2": "Scene description sentence",
"Scene 3": "Scene description sentence",
"Scene 4": "Scene description sentence",
"Scene 5": "Scene description sentence",
"Scene 6": "Scene description sentence"  
"""

In [31]:
GEval_prompt = """
You will be given a news article and summary written for the news article as this format {news article, summary}.

Your task is to rate the summary on one metric.

Please make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.

Evaluation Criteria:

Relevance (1-5) - selection of important content from the source. The summary should include only important information from the source document. 
Annotators were instructed to penalize summaries which contained redundancies and excess information.

Evaluation Steps:

1. Read the summary and the source document carefully.
2. Compare the summary to the source document and identify the main points of the article.
3. Assess how well the summary covers the main points of the article, and how much irrelevant or redundant information it contains.
4. Assign a relevance score from 1 to 5.

Output format: 
{score}
"""

In [10]:
client = OpenAI()

In [11]:
model_id = "stabilityai/stable-diffusion-3.5-large"

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model_nf4 = SD3Transformer2DModel.from_pretrained(
    model_id,
    subfolder="transformer",
    quantization_config=nf4_config,
    torch_dtype=torch.bfloat16
)

pipeline = StableDiffusion3Pipeline.from_pretrained(
    model_id, 
    transformer=model_nf4,
    torch_dtype=torch.bfloat16
)
pipeline.enable_model_cpu_offload()

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [62]:
for index, row in df.iterrows():
    # 폴더 생성
    folder_name = f"{index}"
    os.makedirs(folder_name, exist_ok=True)  # 폴더가 이미 존재하면 무시
    
    # GPT 요청
    input_text = ""
    input_text += "article: " + row['article']
    prompt = storyboard_prompt
    
    response = client.chat.completions.create(
        model= "gpt-4o-mini",
        messages= [
            {
                "role": "system",
                "content": [{ "type": "text", "text": prompt }]
            },
            {
                "role": "user",
                "content": [{ "type": "text", "text": input_text }]
            }
        ],
        max_tokens = 300,
        temperature = 0.0,
        top_p = 0.3,
    )  
    # GPT 결과 파싱 및 저장
    result = response.choices[0].message.content
    # Clean the result string and convert it into a valid JSON format
    cleaned_result = "{" + result.replace("\n", ",").replace('",', '"').replace(",}", "}") + "}"
    summary = ""
    try:
        # Parse the cleaned string into a dictionary
        data = json.loads(cleaned_result)

        # Extract each scene
        scene1 = data.get("Scene 1", "")
        scene2 = data.get("Scene 2", "")
        scene3 = data.get("Scene 3", "")
        scene4 = data.get("Scene 4", "")
        scene5 = data.get("Scene 5", "")
        scene6 = data.get("Scene 6", "")
    except json.JSONDecodeError as e:
        print("Failed to decode JSON:", e)
    summary += scene1
    summary += scene2
    summary += scene3
    summary += scene4
    summary += scene5
    summary += scene6
    print(summary)
    
    gpt_result_path = os.path.join(folder_name, "gpt 요약.json")
    with open(gpt_result_path, "w", encoding="utf-8") as json_file:
        json.dump(summary, json_file, ensure_ascii=False, indent=4)
        
    prompt = GEval_prompt
    article_text = """
    article: 
    """
    article_text += row["article"]
    summary_text = """
    summary: 
    """
    summary_text += summary
    input_text = article_text +"\n"+ summary_text
    response = client.chat.completions.create(
        model= "gpt-4o-mini",
        messages= [
            {
                "role": "system",
                "content": [{ "type": "text", "text": prompt }]
            },
            {
                "role": "user",
                "content": [{ "type": "text", "text": input_text }]
            }
        ],
        max_tokens = 300,
        temperature = 1.0,
        top_p = 1.0,
    )  
    sentences = summary.split(".")
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    
    # GPT 결과 파싱 및 저장
    result = response.choices[0].message.content
    score = result[1]
    print("score: ", score)
    
    if score > '2':
        # 이미지 생성 및 저장
        for i, scene in enumerate(sentences):
            # 이미지 생성 프롬프트     
            print(scene) 
            image = pipeline(
                prompt=scene,
                num_inference_steps=28,
                guidance_scale=4.5,
                max_sequence_length=512,
            ).images[0]
        
            # 이미지 저장
            image_path = os.path.join(folder_name, f"{i}.png")
            image.save(image_path)
    else:
        continue

Daniel Radcliffe stands in front of a large birthday cake with the number 18 on it surrounded by colorful balloons and decorations in a festive roomDaniel Radcliffe is sitting in a cozy room reading a book with a stack of books and CDs beside him while a window shows a sunny day outsideDaniel Radcliffe is at a casino looking at a roulette table with a thoughtful expression while other players are engaged in the game around himDaniel Radcliffe is at a pub holding a soft drink with friends laughing and enjoying themselves at a table filled with snacksDaniel Radcliffe is on a movie set dressed as Rudyard Kipling in a historical costume while filming a scene for the TV movie My Boy JackDaniel Radcliffe is on stage performing in a dramatic scene from Equus with bright stage lights illuminating his focused expression
score:  1
Soledad O'Brien stands at the entrance of the Miami-Dade pretrial detention facility with a camera crew preparing to enter the building.Judge Steven Leifman walks thro

  0%|          | 0/28 [00:00<?, ?it/s]

Judge Steven Leifman walks through the ninth floor of the jail, pointing out the cells where mentally ill inmates are housed, with guards watching from a distance


  0%|          | 0/28 [00:00<?, ?it/s]

Inmates wearing sleeveless robes are seen lying or sitting in their tiny cells, some appearing agitated and shouting incoherent statements


...

  0%|          | 0/28 [00:00<?, ?it/s]

A historical flashback shows a dark, crowded mental hospital from 200 years ago with patients locked away, contrasting with the current jail environment


  0%|          | 0/28 [00:00<?, ?it/s]

Judge Steven Leifman stands outside a new mental health facility, smiling as he discusses the positive changes for inmates and the community with Soledad O'Brien


  0%|          | 0/28 [00:00<?, ?it/s]